In [3]:
import numpy as np
import pandas as pd

In [4]:
# constants

ALPHA = 1/137
PI = np.pi

In [5]:
element = "Be"

In [10]:
# data from element
periodoc_table = pd.read_csv(r"/Users/tema/PycharmProjects/Physics_AI/data/Periodic Table of Elements.csv")

row = periodoc_table[periodoc_table["Symbol"] == element]
row["Density"].item()

1.85